In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

import numpy as np
import pandas as pd
from PIL import Image, ImageEnhance, ImageFilter
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
model = torchvision.models.resnet18(pretrained=True)

# Modify final layer for CIFAR-10
model.fc = nn.Linear(model.fc.in_features, 10)

# Load ImageNet weights except final layer
model = model.to(device)
model.eval()

for param in model.parameters():
    param.requires_grad = False

C:\Users\admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

test_dataset = torchvision.datasets.CIFAR10(
    root="../data",
    train=False,
    download=True,
    transform=transform
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False
)

class_names = test_dataset.classes

In [ ]:
def add_gaussian_noise(img, severity=0.3):
    arr = np.array(img).astype(np.float32) / 255.0
    noise = np.random.normal(0, severity, arr.shape)
    noisy = np.clip(arr + noise, 0, 1)
    return Image.fromarray((noisy * 255).astype(np.uint8))


def apply_blur(img, radius=2):
    return img.filter(ImageFilter.GaussianBlur(radius=radius))


def low_light(img, factor=0.4):
    enhancer = ImageEnhance.Brightness(img)
    return enhancer.enhance(factor)

In [ ]:
def compute_entropy(probs):
    probs = probs + 1e-12
    return -torch.sum(probs * torch.log(probs)).item()

In [ ]:
def run_inference(loader, degradation=None):
    results = []

    for img, label in tqdm(loader):
        img = img[0]  # remove batch
        label = label.item()

        # Convert tensor → PIL for degradation
        pil_img = transforms.ToPILImage()(img)

        if degradation == "noise":
            pil_img = add_gaussian_noise(pil_img)
        elif degradation == "blur":
            pil_img = apply_blur(pil_img)
        elif degradation == "low_light":
            pil_img = low_light(pil_img)

        # Back to tensor
        img_tensor = transform(pil_img).unsqueeze(0).to(device)

        with torch.no_grad():
            logits = model(img_tensor)
            probs = F.softmax(logits, dim=1)[0]

        confidence = torch.max(probs).item()
        prediction = torch.argmax(probs).item()
        entropy = compute_entropy(probs)
        correct = int(prediction == label)

        results.append({
            "confidence": confidence,
            "entropy": entropy,
            "correct": correct
        })

    return pd.DataFrame(results)

In [ ]:
df_clean = run_inference(test_loader, degradation=None)
df_noise = run_inference(test_loader, degradation="noise")
df_blur = run_inference(test_loader, degradation="blur")
df_lowlight = run_inference(test_loader, degradation="low_light")

100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [18:15<00:00,  9.13it/s]


In [ ]:
def summarize(df):
    return {
        "Accuracy": df["correct"].mean(),
        "Avg Confidence": df["confidence"].mean(),
        "Avg Entropy": df["entropy"].mean()
    }

summary = pd.DataFrame.from_dict({
    "Clean": summarize(df_clean),
    "Gaussian Noise": summarize(df_noise),
    "Blur": summarize(df_blur),
    "Low Light": summarize(df_lowlight)
}, orient="index")

summary

,Accuracy,Avg Confidence,Avg Entropy
Clean,0.1210,0.166914,2.238840
Gaussian Noise,0.0987,0.188046,2.220169
Blur,0.1170,0.162637,2.247266
Low Light,0.1053,0.162520,2.249330


In [2]:
!git status
!git add notebooks/
!git commit -m "Baseline vs degraded evaluation on GPU"
!git push

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
